<a href="https://colab.research.google.com/github/DomizianoScarcelli/big-data-project/blob/nn-model/data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuration

In [1]:
#@title Download necessary libraries
!pip install pyspark -qq
!pip install -U -q PyDrive -qq
!apt install openjdk-8-jdk-headless -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 34 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 122545 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headles

In [2]:
#@title Imports
import os
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly

import pyspark
from pyspark.sql import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, FloatType, LongType
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.ml.linalg import SparseVector, DenseVector

from tqdm.notebook import tqdm
import time
import gc

from google.colab import drive

In [3]:
#@title Set up variables
JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"
GDRIVE_DIR = "/content/drive"
GDRIVE_HOME_DIR = GDRIVE_DIR + "/MyDrive"
GDRIVE_DATA_DIR = GDRIVE_HOME_DIR + "/Big Data/datasets"
DATASET_FILE = os.path.join(GDRIVE_DATA_DIR, "pyspark_friendly_spotify_playlist_dataset")
AUDIO_FEATURES_FILE = os.path.join(GDRIVE_DATA_DIR, "pyspark_track_features")
LITTLE_SLICE_FILE = os.path.join(GDRIVE_DATA_DIR, "little_slice")
SMALL_SLICE_FLIE = os.path.join(GDRIVE_DATA_DIR, "small_slice")
LITTLE_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "little_slice_audio_features")
MICRO_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "micro_slice_audio_features")
SPLITTED_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "splitted_pyspark_track_features")
SAVED_DFS_PATH = os.path.join(GDRIVE_DATA_DIR, "saved_dfs")
RANDOM_SEED = 42 # for reproducibility
os.environ["JAVA_HOME"] = JAVA_HOME
os.environ["PYSPARK_PYTHON"]="python"

In [4]:
#@title Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '12G').\
                set('spark.driver.memory', '12G').\
                set('spark.driver.maxResultSize', '100G').\
                set("spark.executor.extraJavaOptions", "-XX:+UseG1GC").\
                setAppName("PySparkTutorial").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [5]:
drive.mount(GDRIVE_DIR, force_remount=True)

Mounted at /content/drive


## Setup ngrok

In [6]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=bc82f741458357d562fa1d4fa6db2c7537fafbb72c0ba6325466fe03fd432233
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [7]:
!ngrok authtoken 2NVN8kdoOnMVtlDGGWtwsbT5M3Q_2EJv2HE77FEXkz978Qtnq

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [8]:
from pyngrok import ngrok

# Open a ngrok tunnel on the port 4050 where Spark is running
port = '4050'
public_url = ngrok.connect(port).public_url

ERROR:pyngrok.process.ngrok:t=2023-05-26T20:16:28+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session obj=csess id=2195eb66bdc7 err="authentication failed: Your account is limited to 1 simultaneous ngrok agent session.\nYou can run multiple tunnels on a single agent session using a configuration file.\nTo learn more, see https://ngrok.com/docs/ngrok-agent/config/\n\nActive ngrok agent sessions in region 'us':\n  - ts_2QLQQj5bi22jVFUZRsZGNafHuNN (34.125.103.177)\r\n\r\nERR_NGROK_108\r\n"
ERROR:pyngrok.process.ngrok:t=2023-05-26T20:16:28+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent session.\nYou can run multiple tunnels on a single agent session using a configuration file.\nTo learn more, see https://ngrok.com/docs/ngrok-agent/config/\n\nActive ngrok agent sessions in region 'us':\n  - ts_2QLQQj5bi22jVFUZRsZGNafHuNN (34.125.103.177)\r\n\r\nERR_NGROK_108\r\n"
ERROR:pyngrok.process.

PyngrokNgrokError: ignored

In [ ]:
print("To access the Spark Web UI console, please click on the following link to the ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

In [ ]:
#@title Check if everything is ok
spark, sc._conf.getAll()


# Data acquisition

In [9]:
from pyspark.ml.linalg import VectorUDT
song_schema = StructType([
    StructField("pos", IntegerType(), True),
    StructField("artist_name", StringType(), True),
    StructField("track_uri", StringType(), True),
    StructField("artist_uri", StringType(), True),
    StructField("track_name", StringType(), True),
    StructField("album_uri", StringType(), True),
    StructField("duration_ms", LongType(), True),
    StructField("album_name", StringType(), True)
])

playlist_schema = StructType([
    StructField("name", StringType(), True),
    StructField("collaborative", StringType(), True),
    StructField("pid", IntegerType(), True),
    StructField("modified_at", IntegerType(), True),
    StructField("num_tracks", IntegerType(), True),
    StructField("num_albums", IntegerType(), True),
    StructField("num_followers", IntegerType(), True),
    StructField("tracks", ArrayType(song_schema), True),
    StructField("num_edits", IntegerType(), True),
    StructField("duration_ms", IntegerType(), True),
    StructField("num_artists", IntegerType(), True),
])

playlist_schema_mapped = StructType([
    StructField("name", StringType(), True),
    StructField("collaborative", StringType(), True),
    StructField("pid", IntegerType(), True),
    StructField("modified_at", IntegerType(), True),
    StructField("num_tracks", IntegerType(), True),
    StructField("num_albums", IntegerType(), True),
    StructField("num_followers", IntegerType(), True),
    StructField("tracks", VectorUDT(), True),
    StructField("num_edits", IntegerType(), True),
    StructField("duration_ms", IntegerType(), True),
    StructField("num_artists", IntegerType(), True),
])

audio_features_schema = StructType([
    StructField("danceability", FloatType(), True),
    StructField("energy", FloatType(), True),
    StructField("key", IntegerType(), True),
    StructField("loudness", FloatType(), True),
    StructField("mode", IntegerType(), True),
    StructField("speechiness", FloatType(), True),
    StructField("acousticness", FloatType(), True),
    StructField("instrumentalness", FloatType(), True),
    StructField("liveness", FloatType(), True),
    StructField("valence", FloatType(), True),
    StructField("tempo", FloatType(), True),
    StructField("type", StringType(), True),
    StructField("id", StringType(), True),
    StructField("uri", StringType(), True),
    StructField("track_href", StringType(), True),
    StructField("analysis_url", StringType(), True),
    StructField("duration_ms", LongType(), True),
    StructField("time_signature", IntegerType(), True)
])

In [10]:
playlist_df = spark.read.schema(playlist_schema).json(DATASET_FILE, multiLine=True)
slice_df = spark.read.schema(playlist_schema).json(SMALL_SLICE_FLIE, multiLine=True)
audio_df = spark.read.schema(audio_features_schema).json(SPLITTED_SLICE_AUDIO_FEATURES, multiLine=True) #has less songs than expected

In [11]:
# slice_df = slice_df.limit(10_000).cache()
NUM_PLAYLISTS = slice_df.count()

## Dataset Train-Test split

In [12]:
from sklearn.model_selection import train_test_split as sklearn_split

def train_test_split(playlist: Row) -> Tuple[Row, Row]:
    train_rows, test_rows = sklearn_split(playlist.tracks, random_state=42)

    playlist_train =  Row(
            name=playlist.name,
            collaborative=playlist.collaborative,
            pid=playlist.pid,
            modified_at=playlist.modified_at,
            num_tracks=playlist.num_tracks,
            num_albums=playlist.num_albums,
            num_followers=playlist.num_followers,
            tracks=train_rows,
            num_edits=playlist.num_edits,
            duration_ms=playlist.duration_ms,
            num_artists=playlist.num_artists,
        )

    playlist_test = Row(
            name=playlist.name,
            collaborative=playlist.collaborative,
            pid=playlist.pid,
            modified_at=playlist.modified_at,
            num_tracks=playlist.num_tracks,
            num_albums=playlist.num_albums,
            num_followers=playlist.num_followers,
            tracks=test_rows,
            num_edits=playlist.num_edits,
            duration_ms=playlist.duration_ms,
            num_artists=playlist.num_artists,
        )
    
    return playlist_train, playlist_test

In [13]:
from pyspark.sql.functions import udf, struct
import shutil

def divide_whole_dataset(playlist_df: DataFrame) -> Tuple[DataFrame, DataFrame]:
  train_test_split_udf = udf(train_test_split, returnType=ArrayType(StructType(playlist_df.schema.fields)))
  divided_df = playlist_df.withColumn("divided", train_test_split_udf(struct(*playlist_df.columns)))
  train_test_df = divided_df.select(col('divided').getItem(0).alias('train'), col('divided').getItem(1).alias('test'))

  train_df = train_test_df.select("train.*")
  test_df = train_test_df.select("test.*")
  return train_df, test_df

TRAIN_DF_PATH = os.path.join(SAVED_DFS_PATH, f"train_df-{NUM_PLAYLISTS}.json")
TEST_DF_PATH = os.path.join(SAVED_DFS_PATH, f"test_df-{NUM_PLAYLISTS}.json")

if os.path.exists(TRAIN_DF_PATH) and os.path.exists(TEST_DF_PATH):
  train_df = spark.read.schema(playlist_schema).json(TRAIN_DF_PATH)
  test_df = spark.read.schema(playlist_schema).json(TEST_DF_PATH)
else:
  # In order to avoid [PATH_ALREADY_EXISTS] errors. 
  if os.path.exists(TRAIN_DF_PATH):
    shutil.rmtree(TRAIN_DF_PATH)
  if os.path.exists(TEST_DF_PATH):
    shutil.rmtree(TEST_DF_PATH)

  train_df, test_df = divide_whole_dataset(slice_df)
  train_df, test_df = train_df.cache(), test_df.cache()
  train_df.write.mode("overwrite").json(TRAIN_DF_PATH)
  test_df.write.mode("overwrite").json(TEST_DF_PATH)

# Data Preparation

In [14]:
from typing import Tuple
from pyspark.sql import DataFrame
from pyspark.ml.linalg import VectorUDT
from pyspark.sql.functions import col, udf, explode
from pyspark.sql.types import IntegerType, ArrayType
from functools import reduce
import pickle

### Get the artists vector

In [16]:
def get_all_artists(playlist_df: DataFrame) -> Tuple[DataFrame, int]:
  """
  Given a playlist dataframe, returns the dataframe containing the unique list of artist_uri that are present in all the playlists.
  """
  all_songs = playlist_df.select(explode("tracks.artist_uri").alias("artist_uri")).distinct()
  return all_songs

def create_artists_pos_mapping(playlist_df: DataFrame) -> DataFrame:
  """
  Given the dataframe of artists_uri, it returns a mapping pos -> artist_uri in order to map each artist_uri to a position inside the embedding vector
  """
  artists_df = get_all_artists(playlist_df)
  artists_df.createOrReplaceTempView("ARTISTS")
  artists_df = spark.sql("""
  SELECT 
      row_number() OVER (
          PARTITION BY '' 
          ORDER BY '' 
      ) as pos,
      *
  FROM 
      ARTISTS
  """)

  artists_df = artists_df.sort("artist_uri")

  ARTIST_VECTOR_LENGTH = artists_df.count()

  return artists_df, ARTIST_VECTOR_LENGTH


def create_artists_vector(playlist_df: DataFrame, mapping: DataFrame, artist_uri_to_id: dict, vector_length) -> DataFrame:
    """
    Returns a DataFrames containing the playlists, but the tracks are represented as a binary sparse vector.
    """

    @udf(returnType=VectorUDT())
    def extract_vector(tracks):
      pos_list = set()

      def reduce_fn(pos_list, row):
          pos_list.add(artist_uri_to_id.get(row.artist_uri))
          return pos_list
      
      pos_list = reduce(reduce_fn, tracks, pos_list)
      
      return SparseVector(vector_length + 1, sorted(list(pos_list)), [1 for _ in pos_list])

    # Apply the mapping UDF on the "tracks" column of the slice_df dataframe
    mapped_df = playlist_df.withColumn('tracks', extract_vector(col('tracks')))

    return mapped_df

def artist_pipeline(playlist_df: DataFrame, is_train: bool = True) -> Tuple[DataFrame, int]:
  artists_df, ARTIST_VECTOR_LENGTH = create_artists_pos_mapping(train_df)
  artist_uri_to_id = artists_df.select('artist_uri', 'pos').rdd.collectAsMap() # TODO: Pass it as a parameter maybe?

  train_string = "train" if is_train else "test"

  ARTISTS_EMBEDDINGS = os.path.join(SAVED_DFS_PATH, f"artists_embeddings-{train_string}-{NUM_PLAYLISTS}.json")
  ARTIST_VECTOR_LENGTH_PATH = os.path.join(SAVED_DFS_PATH, f"artist_vector_length-{train_string}-{NUM_PLAYLISTS}.txt")
  with open(ARTIST_VECTOR_LENGTH_PATH, "w") as f:
    f.write('%d' % ARTIST_VECTOR_LENGTH)
  artists_slice_df = create_artists_vector(train_df, 
                                           artists_df, 
                                           artist_uri_to_id, 
                                           ARTIST_VECTOR_LENGTH).cache()
  artists_slice_df.write.mode("overwrite").json(ARTISTS_EMBEDDINGS)
  
  return artists_slice_df, ARTIST_VECTOR_LENGTH

artist_slice_df_train, ARTIST_VECTOR_LENGTH_TRAIN = artist_pipeline(train_df)
artist_slice_df_test, ARTIST_VECTOR_LENGTH_TEST = artist_pipeline(test_df, is_train=False)

## Get the tracks vector

In [17]:
def get_all_songs(playlist_df: DataFrame) -> DataFrame:
  """
  Given a playlist dataframe, returns the dataframe containing the unique list of track_uri that are present in all the playlists.
  """
  all_songs = playlist_df.select(explode("tracks.track_uri").alias("track_uri")).distinct()
  return all_songs

def create_songs_pos_mapping(playlist_df: DataFrame) -> Tuple[DataFrame, int]:
  """
  Given the dataframe of tracks_uris, it returns a mapping pos -> track_uri in order to map each track_uri to a position inside the embedding vector
  """
  songs_df = get_all_songs(playlist_df)
  songs_df.createOrReplaceTempView("SONGS_INFO")

  songs_info_df = spark.sql("""
  SELECT 
      row_number() OVER (
          PARTITION BY '' 
          ORDER BY '' 
      ) as pos,
      *
  FROM 
      SONGS_INFO
  """)

  songs_df = songs_info_df.sort("track_uri")
  RATING_VECTOR_LENGTH = songs_df.count()

  return songs_df, songs_info_df, RATING_VECTOR_LENGTH


#TODO: Since the .rdd is very slow, I can embed the position information of the track inside the track itself,
# So then I can just do pos_list.add(row.rating_position) in a few miliseconds. 
def map_track_df_to_pos(playlist_df: DataFrame, mapping: DataFrame, track_uri_to_id: dict, vector_length: int) -> DataFrame:
    """
    Returns a DataFrames containing the playlists, but the tracks are represented as a binary sparse vector.
    """

    @udf(returnType=VectorUDT())
    def extract_vector(tracks):
      pos_list = set()

      def reduce_fn(pos_list, row):
          pos_list.add(track_uri_to_id.get(row.track_uri))
          return pos_list
      
      pos_list = reduce(reduce_fn, tracks, pos_list)
      
      return SparseVector(vector_length + 1, sorted(list(pos_list)), [1 for _ in pos_list])

    # Apply the mapping UDF on the "tracks" column of the slice_df dataframe
    mapped_df = playlist_df.withColumn('tracks', extract_vector(col('tracks')))

    return mapped_df

def track_pipeline(playlist_df: DataFrame, is_train: bool = True) -> Tuple[DataFrame, DataFrame, int]:
  songs_df, songs_info_df, RATING_VECTOR_LENGTH = create_songs_pos_mapping(train_df)

  track_uri_to_id = songs_df.select('track_uri', 'pos').rdd.collectAsMap()
  songs_slice_df = map_track_df_to_pos(train_df, 
                                       songs_df, 
                                       track_uri_to_id, 
                                       RATING_VECTOR_LENGTH).cache()

  train_string = "train" if is_train else "test"

  SONGS_EMBEDDINGS = os.path.join(SAVED_DFS_PATH, f"songs_embeddings-{train_string}-{NUM_PLAYLISTS}.json")
  SONGS_INFO_DF = os.path.join(SAVED_DFS_PATH, f"songs_info_df-{train_string}-{NUM_PLAYLISTS}.json")
  RATING_VECTOR_LENGTH_PATH = os.path.join(SAVED_DFS_PATH, f"songs_vector_length-{train_string}-{NUM_PLAYLISTS}.txt")
  with open(RATING_VECTOR_LENGTH_PATH, "w") as f:
    f.write('%d' % RATING_VECTOR_LENGTH)
  songs_slice_df.write.mode("overwrite").json(SONGS_EMBEDDINGS)
  songs_info_df.write.mode("overwrite").json(SONGS_INFO_DF)
  return songs_slice_df, songs_info_df, RATING_VECTOR_LENGTH

songs_slice_df_train, songs_info_df_train, SONGS_VECTOR_LENGTH_TRAIN = track_pipeline(train_df)
songs_slice_df_train, songs_info_df_train, SONGS_VECTOR_LENGTH_TEST = track_pipeline(test_df, is_train=False)